In [4]:
!pip install "yfinance"==0.2.59

'''esse script só funcionará com essa versão específica do yfinance, qualquer versão superior incorrerá no erro "too many requestes" 
no momento de fazer o download dos dados dos tickers'''

'esse script só funcionará com essa versão específica do yfinance, qualquer versão superior incorrerá no erro "too many requestes" \nno momento de fazer o download dos dados dos tickers'

In [5]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date
from flask import Flask, jsonify




In [7]:
# Defina o período de tempo para as cotações
start_date = '2021-01-01'
end_date = date.today().strftime('%Y-%m-%d')

# Obtendo informação de ativos na bolsa
ticker_fundos = ['RECR11.SA', 'VGIA11.SA', 'XPLG11.SA', 'VGIR11.SA', 'HGLG11.SA', 'CPTS11.SA', 'XPSF11.SA', 'GAME11.SA', 'BBDC4.SA', 'MAXR11.SA', 'MXRF11.SA', 'WEGE3.SA', 'CPLE3.SA',  'ITUB3.SA','BBAS3.SA','SAPR11.SA','PETR4.SA','PETR3.SA','MGLU3.SA','VALE3.SA','ITSA4.SA','GOAU4.SA','ELET6.SA','ELET3.SA','EMBR3.SA','GGBR4.SA','NTCO3.SA','KLBN11.SA','SANB11.SA','TAEE11.SA','TOTS3.SA']

# Download dos dados
cotacoes_fundos = yf.download(ticker_fundos, actions=True, start=start_date, end=end_date)

[*********************100%***********************]  31 of 31 completed


In [8]:
# Resetar o índice e rearranjar colunas
cotacoes = cotacoes_fundos.stack(level=1).reset_index()
cotacoes = cotacoes.rename(columns={'level_1': 'Ticker'})

print(cotacoes)
cotacoes.to_excel('dados_fundos.xlsx', index=False)

C:\Users\Adriel\AppData\Local\Temp\ipykernel_9884\3453741512.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  cotacoes = cotacoes_fundos.stack(level=1).reset_index()


Price       Date     Ticker      Close  Dividends       High        Low  \
0     2021-01-04   BBAS3.SA  13.532604        0.0  14.144614  13.500204   
1     2021-01-04   BBDC4.SA  16.245663        0.0  16.921796  16.141168   
2     2021-01-04   CPLE3.SA   4.991761        0.0   5.122202   4.930548   
3     2021-01-04  CPTS11.SA   0.210110        0.0   0.212515   0.209193   
4     2021-01-04   ELET3.SA  30.496195        0.0  32.011162  30.325012   
...          ...        ...        ...        ...        ...        ...   
35088 2025-08-15  VGIA11.SA   8.990000        0.0   9.010000   8.950000   
35089 2025-08-15  VGIR11.SA   9.470000        0.0   9.480000   9.430000   
35090 2025-08-15   WEGE3.SA  35.970001        0.0  36.799999  35.720001   
35091 2025-08-15  XPLG11.SA  96.910004        0.0  98.300003  96.029999   
35092 2025-08-15  XPSF11.SA   5.990000        0.0   5.990000   5.900000   

Price       Open  Stock Splits      Volume  
0      14.123013           0.0  27749200.0  
1      16

In [9]:

#Removendo sulfixo ".SA" do final das strings na coluna Ticker
cotacoes["Ticker"] = cotacoes["Ticker"].str.rstrip(".SA")
print(f'Tabela cotações \n {cotacoes}')
print(cotacoes.head(0))

Tabela cotações 
 Price       Date  Ticker      Close  Dividends       High        Low  \
0     2021-01-04   BBAS3  13.532604        0.0  14.144614  13.500204   
1     2021-01-04   BBDC4  16.245663        0.0  16.921796  16.141168   
2     2021-01-04   CPLE3   4.991761        0.0   5.122202   4.930548   
3     2021-01-04  CPTS11   0.210110        0.0   0.212515   0.209193   
4     2021-01-04   ELET3  30.496195        0.0  32.011162  30.325012   
...          ...     ...        ...        ...        ...        ...   
35088 2025-08-15  VGIA11   8.990000        0.0   9.010000   8.950000   
35089 2025-08-15  VGIR11   9.470000        0.0   9.480000   9.430000   
35090 2025-08-15   WEGE3  35.970001        0.0  36.799999  35.720001   
35091 2025-08-15  XPLG11  96.910004        0.0  98.300003  96.029999   
35092 2025-08-15  XPSF11   5.990000        0.0   5.990000   5.900000   

Price       Open  Stock Splits      Volume  
0      14.123013           0.0  27749200.0  
1      16.921796           

In [10]:
# Criando a tabela Dividends a partir da tabela cotacoes_stacked sendo que só entrarão linhas em que o dividendo for > 0
dividends = cotacoes[cotacoes['Dividends'] > 0]
print("Tabela filtrada")
print(type(dividends))

Tabela filtrada
<class 'pandas.core.frame.DataFrame'>


In [11]:
#Removendo as colunas redundantes de ambas as tabelas
dividends = dividends.drop(['Close', 'High', 'Low', 'Open', 'Stock Splits', 'Volume'],axis=1)
cotacoes = cotacoes.drop(['Dividends', 'Stock Splits'], axis=1)
print(f'Tabela cotações \n {dividends}')


Tabela cotações 
 Price       Date  Ticker  Dividends
11    2021-01-04   ITUB3   0.013636
20    2021-01-04  SAPR11   0.488226
29    2021-01-05   BBDC4   0.015681
153   2021-01-11  MAXR11  11.379252
159   2021-01-11  SANB11   0.178421
...          ...     ...        ...
34982 2025-08-12   GOAU4   0.080000
35006 2025-08-13  CPTS11   0.086000
35027 2025-08-13   VALE3   1.895387
35028 2025-08-13  VGIA11   0.130000
35029 2025-08-13  VGIR11   0.140000

[888 rows x 3 columns]


In [13]:
#Cálculo de métricas financeieras:

def calculo_de_metricas(cotacoes):
    # Médias móveis simples (SMA)
    sma_windows = [7, 30]
    for w in sma_windows:
        cotacoes[f"SMA_{w}"] = (
            cotacoes.groupby("Ticker")["Close"]
            .transform(lambda x, w=w: x.rolling(window=w).mean())
        )

    # Médias móveis exponenciais (EMA)
    ema_spans = [7, 30]
    for s in ema_spans:
        cotacoes[f"EMA_{s}"] = (
            cotacoes.groupby("Ticker")["Close"]
            .transform(lambda x, s=s: x.ewm(span=s, adjust=False).mean())
        )

    # Variações percentuais
    changes = {
        "Daily Change (%)": lambda df: (df["Close"] - df["Open"]) / df["Open"] * 100,
        "Weekly Change (%)": lambda df: df.groupby("Ticker")["Close"].pct_change(5) * 100,
        "Monthly Change (%)": lambda df: df.groupby("Ticker")["Close"].pct_change(21) * 100,
    }
    for name, func in changes.items():
        cotacoes[name] = func(cotacoes)

    # Volume médio
    vol_windows = [7, 30]
    for w in vol_windows:
        cotacoes[f"Vol_Media_{w}"] = (
            cotacoes.groupby("Ticker")["Volume"]
            .transform(lambda x, w=w: x.rolling(window=w).mean())
        )

    # Liquidez
    cotacoes["Liquidez"] = cotacoes["Vol_Media_30"]

    return cotacoes


In [14]:
cotacoes = calcular_metricas(cotacoes)

print('métricas calculadas e inseridas no df cotacoes')
display(cotacoes)

métricas calculadas e inseridas no df cotacoes


Price,Date,Ticker,Close,High,Low,Open,Volume,Daily Change (%),Weekly Change (%),Monthly Change (%),SMA_7,SMA_30,EMA_7,EMA_30,Vol_Media_7,Vol_Media_30,Liquidez
0,2021-01-04,BBAS3,13.532604,14.144614,13.500204,14.123013,27749200.0,-4.180473,NaN,NaN,NaN,NaN,13.532604,13.532604,NaN,NaN,NaN
1,2021-01-04,BBDC4,16.245663,16.921796,16.141168,16.921796,48196357.0,-3.995637,NaN,NaN,NaN,NaN,16.245663,16.245663,NaN,NaN,NaN
2,2021-01-04,CPLE3,4.991761,5.122202,4.930548,5.101798,2108000.0,-2.156835,NaN,NaN,NaN,NaN,4.991761,4.991761,NaN,NaN,NaN
3,2021-01-04,CPTS11,0.210110,0.212515,0.209193,0.212311,572410.0,-1.036466,NaN,NaN,NaN,NaN,0.210110,0.210110,NaN,NaN,NaN
4,2021-01-04,ELET3,30.496195,32.011162,30.325012,31.703033,4180700.0,-3.806695,NaN,NaN,NaN,NaN,30.496195,30.496195,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35088,2025-08-15,VGIA11,8.990000,9.010000,8.950000,9.010000,123417.0,-0.221981,-0.430338,0.891377,8.975428,8.901352,8.953374,8.879443,1.535990e+05,1.644617e+05,1.644617e+05
35089,2025-08-15,VGIR11,9.470000,9.480000,9.430000,9.450000,265059.0,0.211645,0.115026,-0.403690,9.424160,9.401193,9.425375,9.388386,4.556301e+05,5.545118e+05,5.545118e+05
35090,2025-08-15,WEGE3,35.970001,36.799999,35.720001,36.799999,7177700.0,-2.255429,-5.590544,-14.133248,37.144286,38.505011,36.785641,38.401829,8.412386e+06,1.198035e+07,1.198035e+07
35091,2025-08-15,XPLG11,96.910004,98.300003,96.029999,97.699997,39796.0,-0.808591,0.633438,-2.064895,96.700001,97.617458,96.886232,97.128493,2.535629e+04,2.840117e+04,2.840117e+04


In [15]:
#exportando as tabelas em XLSX
cotacoes.to_excel('dados_fundos.xlsx', index=False)
dividends.to_excel('Dividendos.xlsx', index=False)
print("Dados exportados para 'dados_fundos.xlsx'")

Dados exportados para 'dados_fundos.xlsx'
